In [1]:
import pandas as pd
import numpy as np
import win32com.client
import time

In [47]:
file = "C:/Users\lukec\OneDrive - University of Cambridge\Projects\PhD\Data\IHS\Ethylene\ETHYLENE FROM WIDE RANGE NAPHTHA, MAXIMUM ETHYLENE, FRONT END DEMETHANIZER - Germany.xlsx"
footer = 11
headerLines = 5
colSep = 6

In [52]:
def refresh_excel_file(file:str, pause=False):
    # Refresh Excel file to have numbers for formula cells
    xlapp = win32com.client.DispatchEx("Excel.Application")
    wb = xlapp.Workbooks.Open(file)
    wb.RefreshAll()
    if pause: time.sleep(pause)
    wb.Save()
    wb.Close()
    xlapp.Quit()
    return file

In [167]:
def extract_properties(df):
    properties = list(map(lambda x: x.strip(), df.loc[2][0].split(",")))+['Name: '+df.loc[1][0]]
    properties = dict(map(lambda x: x.split(": "), properties))
    return properties

def add_property_columns(df, properties):
    print(list(properties.values()))
    df[list(properties.keys())] = list(properties.values())
    return df

def excel_multilayer_table_extract(df, dataType='Attribute', header=0, colStart=0, colEnd=-1, properties=False):
    data = df[header:][df.columns[colStart:colEnd]]
    data.columns = [dataType] + list(data.iloc[0][1:].values)
    data.columns = list(map(lambda x: 'Unit' if x is np.nan else x, data.columns.values))
    data.dropna(subset=[data.columns[0]], inplace=True)

    typesInd = [0] + list(np.where(data.index[1:] - data.index[:-1] != 1)[0]+1)
    data['Type'] = list(map(lambda x: data[dataType].values[typesInd[np.sum(np.array(typesInd) < x+1)-1]], range(len(data))))
    data.dropna(subset=data.columns[1:-1], how='all', inplace=True)
    data.reset_index(drop=True, inplace=True)
    if properties: data = add_property_columns(data, properties)
    return data

In [171]:
# Import sheet
sheet = pd.read_excel(refresh_excel_file(file),skipfooter=footer)
properties = extract_properties(sheet)
materials = excel_multilayer_table_extract(sheet, dataType='Material', header=headerLines, colEnd=colSep,
                                           properties=properties)
cost = excel_multilayer_table_extract(sheet, dataType='Cost', header=headerLines, colStart=colSep+1,
                                      properties=properties)

['Germany', 'ETHYLENE', '1087', '135.83 ¢/KG', '2M-1622', '1.06 (1US $ = 0.85)', '2021 Q3', '2014', 'ETHYLENE FROM WIDE RANGE NAPHTHA, MAXIMUM ETHYLENE, FRONT END DEMETHANIZER']


In [127]:
from pandas_xlsx_tables import df_to_xlsx_table
df_to_xlsx_table(cost, "Costs")

In [ ]:
#!pip install pivottablejs
# from pivottablejs import pivot_ui
# pivot_ui(cost,outfile_path='pivottablejs.html')